In [44]:
import numpy as np
import matplotlib.pyplot as plt
import datasets
import matplotlib.transforms as mtransforms
import evaluate
import pandas as pd
from scipy.stats import wasserstein_distance as wass
import pickle as pkl
from evaluate import load
from datasets import load_dataset, DatasetDict, load_from_disk
import os
from time import sleep
from tqdm.auto import tqdm
from natsort import natsorted

In [91]:
def get_bertscore(data):
    
    highlights = []
    model_s = []


    # for j in data['highlights']:
    #     highlights.append(' '.join(j))

    # for k in data['model_summaries']:
    #     model_s.append(' '.join(k))
    
    bertscore = load("bertscore")
    
    results = bertscore.compute(predictions=data['model_summaries'], references=data['highlights'], lang="en", device='cuda:1')
    mean_precision=sum(results['precision'])/len(results['precision'])
    mean_recall=sum(results['recall'])/len(results['recall'])
    mean_f1=sum(results['f1'])/len(results['f1'])
    
    return mean_precision,mean_recall,mean_f1

In [92]:
def get_rouge1(data):

    highlights = []
    model_s = []


    # for j in data['highlights']:
    #     highlights.append(' '.join(j))

    # for k in data['model_summaries']:
    #     model_s.append(' '.join(k))


    rouge = evaluate.load('rouge')

    print("==> Comparing generated summaries with gold summaries")
    results = rouge.compute(predictions=data['model_summaries'], references=data['highlights'])
    
    return results['rouge1']

In [93]:
def get_rouge2(data):

    highlights = []
    model_s = []


    # for j in data['highlights']:
    #     highlights.append(' '.join(j))

    # for k in data['model_summaries']:
    #     model_s.append(' '.join(k))


    rouge = evaluate.load('rouge')

    print("==> Comparing generated summaries with gold summaries")
    results = rouge.compute(predictions=data['model_summaries'], references=data['highlights'])
    
    return results['rouge2']

In [94]:
def get_rougeL(data):

    highlights = []
    model_s = []


    # for j in data['highlights']:
    #     highlights.append(' '.join(j))

    # for k in data['model_summaries']:
    #     model_s.append(' '.join(k))


    rouge = evaluate.load('rouge')

    print("==> Comparing generated summaries with gold summaries")
    results = rouge.compute(predictions=data['model_summaries'], references=data['highlights'])
    
    return results['rougeL']

In [63]:
with open('results-xsum.pkl', 'rb') as f:
    xsum_original=pkl.load(f)

In [64]:
with open('results-paraphrased-xsum.pkl', 'rb') as f:
    xsum_paraphrased=pkl.load(f)

In [95]:
with open('results.pkl', 'rb') as f:
    cnn_original=pkl.load(f)

In [96]:
with open('results-paraphrased-cnn.pkl', 'rb') as f:
    cnn_paraphrased=pkl.load(f)

In [58]:
len(cnn_paraphrased)

100

In [137]:
xsum_original_first=[]
for items in cnn_original:
    xsum_original_first.append(items[1])

In [138]:
xsum_paraphrased_first=[]
for items in cnn_paraphrased:
    xsum_paraphrased_first.append(items[1])

In [99]:
xsum_original_first[0]

"1. The Palestinian Authority has officially become a member of the International Criminal Court, granting the court jurisdiction over alleged crimes committed in Palestinian territories.\n\n2. This move has been met with opposition from Israel and the United States, who both opposed the Palestinians' membership in the ICC.\n\n3. The ICC's preliminary examination will review evidence and determine whether to investigate war crimes committed by both Israelis and Palestinians since June 2014."

In [100]:
xsum_paraphrased_first[0]

'1. The Palestinian Authority has become a member of the International Criminal Court, giving the court jurisdiction to investigate alleged crimes in the Palestinian territories.'

In [139]:
pkls=os.listdir('../../paraphrased_articles/cnn')
pkls=natsorted(pkls)
pkls_list=[]
count=0
bad_index=[]
for k,pikl in enumerate(pkls):
    with open('../../paraphrased_articles/cnn/{}'.format(pikl),'rb') as f:
        file=pkl.load(f)
    if not file and k<100:
        bad_index.append(k)
        count+=1
        file.append(' ') #no paraphrasing possible
    pkls_list.extend(file)

# print(count)

In [140]:
# data_llama_cnn.select(i for i in range(len(data_llama_cnn)) 
#                  if i not in set(bad_index))

In [141]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'document'
summary_key = 'highlights'
data_original=dataset['test']
data_original=data_original.select(range(100))
data_original = data_original.add_column('model_summaries', xsum_original_first)
#data[article_key]=pkls_list
data_original.select(i for i in range(len(data_original)) 
                 if i not in set(bad_index))



Dataset({
    features: ['article', 'highlights', 'id', 'model_summaries'],
    num_rows: 96
})

In [142]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'document'
summary_key = 'highlights'
data_paraphrased=dataset['test']
data_paraphrased=data_paraphrased.select(range(100))
data_paraphrased = data_paraphrased.add_column('model_summaries', xsum_paraphrased_first)
#data[article_key]=pkls_list
data_paraphrased.select(i for i in range(len(data_paraphrased)) 
                 if i not in set(bad_index))



Dataset({
    features: ['article', 'highlights', 'id', 'model_summaries'],
    num_rows: 96
})

In [147]:
get_bertscore(data_original)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(0.8582734239101409, 0.8844329518079758, 0.871051213145256)

In [148]:
get_rouge1(data_original)

==> Comparing generated summaries with gold summaries


0.2850808951574686

In [149]:
get_rouge2(data_original)

==> Comparing generated summaries with gold summaries


0.107321159671523

In [150]:
get_rougeL(data_original)

==> Comparing generated summaries with gold summaries


0.20249389712647695